In [1]:
import asyncio
import websockets
import json
import time

import sys
sys.path.append("./chainklik")
import pandas as pd
import requests
from web3 import Web3
from collections import defaultdict 

import libs.common.utils as utils
import libs.common.payload as payload
import config.config as cfg
from nodes.data import *
from nodes.eth import *

In [2]:
# Connect to an Ethereum node
w3 = Web3(Web3.HTTPProvider(cfg.config["eth_sepolia_http_url"]))

# Set sender and recipient addresses
sender_address = '0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7'
recipient_address = '0xaf6667a2F847beeca6a6604126Dc28344518840b'

# Set private key for the sender's account. 
private_key = '1dab201501e8b882ca3413edcdfed263e4834cd8ea4c9586aca7fb699c51d681'

In [3]:
balance_sender = w3.from_wei(w3.eth.get_balance(sender_address), 'ether')
balance_recipient = w3.from_wei(w3.eth.get_balance(recipient_address), 'ether')

print(f'The balance of { sender_address } is: { balance_sender } ETH')
print(f'The balance of { recipient_address } is: { balance_recipient } ETH')

The balance of 0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7 is: 0.376363494037823 ETH
The balance of 0xaf6667a2F847beeca6a6604126Dc28344518840b is: 0.1065 ETH


In [4]:
w3.eth.get_block("latest")["number"]

5027821

In [5]:
bt = BlockTimer(w3.eth.get_block("latest")["number"], w3.eth.get_block("latest")["number"]+20, 5)
data_node = DataNode("data", {}, {}, bt)

In [6]:
tx_node1 = TxNode("tx1",{"trigger":"data"},{"wallet":sender_address,"to":recipient_address,"amount":0.001}, w3, private_key)
tx_node2 = TxNode("tx2",{"trigger":"data"},{"wallet":sender_address,"to":recipient_address,"amount":0.002}, w3, private_key)
tx_node3 = TxNode("tx3",{"trigger":"tx1"},{"wallet":sender_address,"to":recipient_address,"amount":0.003}, w3, private_key)

In [7]:
node_layers = [
    ["data"],
    ["tx1", "tx2"],
    ["tx3"]
]

In [8]:
nodes = {
    "data": data_node, 
    "tx1": tx_node1,
    "tx2": tx_node2,
    "tx3": tx_node3
}

In [9]:
nodes["data"].__dict__

{'id': 'data',
 'deps': {},
 'params': {},
 'output': None,
 'active': False,
 'finalized': True,
 'timer': <nodes.decorator.BlockTimer at 0x11e881110>}

In [10]:
def print_nodes_state(node_layers):
    print('######## node state ##########')
    for node_layer in node_layers:
        print([(nodes[key].active,nodes[key].finalized) for key in node_layer])

In [11]:
ctx = {}
ctx["pending_txs"] = defaultdict(int)

In [12]:
async def process_message(block_info):
    print(int(block_info["number"], 16))
    ctx["block_time"] = int(block_info["number"], 16)
    values = {"block_time": int(block_info["number"], 16)}
    for key in nodes:
        if nodes[key].finalized:
            nodes[key].active = False
        else:
            nodes[key].active = True

    print_nodes_state(node_layers)
    
    for node_layer in node_layers:
        for key in node_layer:
            print(key)
            if len(nodes[key].deps) == 0: # no deps/input node
                nodes[key].run(ctx, values)
            else:
                for dep in nodes[key].deps:
                    if nodes[nodes[key].deps[dep]].active and nodes[nodes[key].deps[dep]].finalized:
                        nodes[key].run(ctx, values)
                    elif nodes[key].active:
                        nodes[key].run(ctx, values)

    print_nodes_state(node_layers)

    print(bt.__dict__)
    
    # for node_layer in node_layers:
    #     for key in node_layer:
    #         print(nodes[key].__dict__)
    print(ctx)

In [13]:
async def message():
    async with websockets.connect(cfg.config["eth_sepolia_ws_url"]) as ws:
        sub_newheads = { "id":1, "jsonrpc":"2.0", "method":"eth_subscribe", "params":["newHeads"] }
        await ws.send(json.dumps(sub_newheads))
        sub_res = await ws.recv()
        print(json.loads(sub_res))
        while True:
            message = await asyncio.wait_for(ws.recv(), timeout=60)
            block_info = json.loads(message)["params"]["result"]
            # start = time.time()
            await process_message(block_info)
            # end = time.time()
            # print(end-start)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

loop = asyncio.get_event_loop()
while True:
    loop.run_until_complete(message())

{'jsonrpc': '2.0', 'id': 1, 'result': '0x4a13dc7fe597007087637faac90e6774'}
5027822
######## node state ##########
[(False, True)]
[(False, True), (False, True)]
[(False, True)]
data
tx1
New transaction. tx1
Transaction sent! tx1
tx2
New transaction. tx2
Transaction sent! tx2
tx3
######## node state ##########
[(True, True)]
[(True, False), (True, False)]
[(False, True)]
{'start': 5027826, 'end': 5027841, 'frequency': 5}
{'pending_txs': defaultdict(<class 'int'>, {'0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7': 2}), 'block_time': 5027822}
5027823
######## node state ##########
[(False, True)]
[(True, False), (True, False)]
[(False, True)]
data
tx1
tx2
tx3
######## node state ##########
[(False, True)]
[(True, False), (True, False)]
[(False, True)]
{'start': 5027826, 'end': 5027841, 'frequency': 5}
{'pending_txs': defaultdict(<class 'int'>, {'0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7': 2}), 'block_time': 5027823}
5027824
######## node state ##########
[(False, True)]
[(True, False), (Tru